In [20]:
import nltk
import tensorflow as tf
from tensorflow import keras


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
intents =  {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Kush, your bot assistant", "I'm Kush, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}


for intent in intents['intents']:
    for pattern in intent['patterns']:

        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))

        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


training = []

output_empty = [0] * len(classes)

for doc in documents:
    
    bag = []
    
    pattern_words = doc[0]
    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")



model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))
adam=tf.keras.optimizers.legacy.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())

hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


33 documents
8 classes ['about', 'complaint', 'createaccount', 'goodbye', 'greeting', 'help', 'name', 'thanks']
52 unique lemmatized words ["'s", 'a', 'about', 'account', 'an', 'anyone', 'are', 'bye', 'call', 'can', 'complaint', 'could', 'create', 'do', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'name', 'need', 'new', 'open', 'please', 'raise', 'see', 'service', 'should', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'want', 'what', 'whats', 'who', 'you', 'your']
Training data created
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               6784      
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                

C:\Users\kusha\AppData\Local\Temp\ipykernel_12228\4011988318.py:110: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


7/7 [==============================] - 1s 2ms/step - loss: 2.0620 - accuracy: 0.1818
Epoch 2/500
7/7 [==============================] - 0s 3ms/step - loss: 2.0436 - accuracy: 0.1515
Epoch 3/500
7/7 [==============================] - 0s 3ms/step - loss: 1.9698 - accuracy: 0.2424
Epoch 4/500
7/7 [==============================] - 0s 3ms/step - loss: 1.9208 - accuracy: 0.3333
Epoch 5/500
7/7 [==============================] - 0s 3ms/step - loss: 1.8476 - accuracy: 0.4242
Epoch 6/500
7/7 [==============================] - 0s 3ms/step - loss: 1.8592 - accuracy: 0.3333
Epoch 7/500
7/7 [==============================] - 0s 3ms/step - loss: 1.7861 - accuracy: 0.4848
Epoch 8/500
7/7 [==============================] - 0s 3ms/step - loss: 1.7169 - accuracy: 0.5455
Epoch 9/500
7/7 [==============================] - 0s 3ms/step - loss: 1.6808 - accuracy: 0.4545
Epoch 10/500
7/7 [==============================] - 0s 3ms/step - loss: 1.5636 - accuracy: 0.6667
Epoch 11/500
7/7 [=======================

7/7 [==============================] - 0s 2ms/step - loss: 0.0820 - accuracy: 1.0000
Epoch 85/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0451 - accuracy: 1.0000
Epoch 86/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0342 - accuracy: 1.0000
Epoch 87/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0391 - accuracy: 1.0000
Epoch 88/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0555 - accuracy: 1.0000
Epoch 89/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0310 - accuracy: 1.0000
Epoch 90/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0399 - accuracy: 1.0000
Epoch 91/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 92/500
7/7 [==============================] - 0s 3ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 93/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 94/500
7/7 [===============

7/7 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 168/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 169/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0568 - accuracy: 0.9697
Epoch 170/500
7/7 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 171/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 172/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 173/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 174/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 175/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 1.0000
Epoch 176/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 177/500
7/7 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 250/500
7/7 [==============================] - 0s 3ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 251/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 252/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 253/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 254/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 255/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 256/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 257/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 258/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 259/500
7/7 [=====

7/7 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 332/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 1.0000
Epoch 333/500
7/7 [==============================] - 0s 2ms/step - loss: 6.8882e-04 - accuracy: 1.0000
Epoch 334/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 335/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 336/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 337/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 338/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 339/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 340/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 341/500
7/7 [=

7/7 [==============================] - 0s 3ms/step - loss: 6.6262e-04 - accuracy: 1.0000
Epoch 413/500
7/7 [==============================] - 0s 2ms/step - loss: 4.1740e-04 - accuracy: 1.0000
Epoch 414/500
7/7 [==============================] - 0s 2ms/step - loss: 3.0529e-04 - accuracy: 1.0000
Epoch 415/500
7/7 [==============================] - 0s 2ms/step - loss: 5.1233e-05 - accuracy: 1.0000
Epoch 416/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 417/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 418/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 419/500
7/7 [==============================] - 0s 2ms/step - loss: 4.9486e-04 - accuracy: 1.0000
Epoch 420/500
7/7 [==============================] - 0s 2ms/step - loss: 1.5702e-04 - accuracy: 1.0000
Epoch 421/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0023 - accuracy: 1.0000


7/7 [==============================] - 0s 2ms/step - loss: 2.9055e-04 - accuracy: 1.0000
Epoch 493/500
7/7 [==============================] - 0s 2ms/step - loss: 3.7970e-04 - accuracy: 1.0000
Epoch 494/500
7/7 [==============================] - 0s 3ms/step - loss: 2.6956e-04 - accuracy: 1.0000
Epoch 495/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 496/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 497/500
7/7 [==============================] - 0s 2ms/step - loss: 6.4285e-04 - accuracy: 1.0000
Epoch 498/500
7/7 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 499/500
7/7 [==============================] - 0s 2ms/step - loss: 9.4740e-04 - accuracy: 1.0000
Epoch 500/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000
model created


In [22]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    
    sentence_words = nltk.word_tokenize(sentence)
    
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words



def bow(sentence, words, show_details=True):
    
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)


scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )


EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")




scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()